In [1]:
import argparse
import os
import random
import numpy as np
from argparse import Namespace

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from packaging import version
from multiomic_modeling.models.trainer import MultiomicTrainer
from multiomic_modeling.data.data_loader import MultiomicDataset, SubsetRandomSampler, multiomic_dataset_builder

import pytorch_lightning as pl
import torch

if version.parse(pl.__version__) < version.parse("1.0.2"):
    raise RuntimeError("PyTorch Lightning>=1.0.2 is required for this example.")

In [2]:
training_params_global = []
def objective(trial: optuna.trial.Trial) -> float:
    """ Main fonction to poptimize with Optuna """
    model_params = {
        "d_input_enc": 2000,
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "nb_classes_dec": 33,
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "weight_decay": trial.suggest_float("weight_decay", 0.1, 0.5),
        "activation": "relu",
        "optimizer": "Adam",
        "lr_scheduler": "cosine_with_restarts",
        "loss": "ce",
        "n_epochs": 1,
        "batch_size": 128,
        "class_weights":[4.1472332 , 0.87510425, 0.30869373, 1.2229021 , 8.47878788,
            0.7000834 , 7.94886364, 1.87032086, 0.63379644, 0.63169777,
            4.19280719, 0.40417951, 1.08393595, 1.90772727, 0.72125795,
            0.87110834, 0.59523472, 0.61243251, 4.38557994, 0.63169777,
            1.94666048, 2.04035002, 0.67410858, 2.08494784, 1.40791681,
            0.79654583, 0.74666429, 2.74493133, 0.65783699, 3.02813853,
            0.65445189, 6.6937799 , 4.76931818],
        "d_model_enc_dec": trial.suggest_categorical("n_heads_enc_dec", [128, 256, 512, 1024, 2048]),
        "n_heads_enc_dec": trial.suggest_categorical("n_heads_enc_dec", [128, 256, 512, 1024, 2048]),
        "n_layers_enc_dec": trial.suggest_int("n_layers_enc_dec", 1, 10, step=1)
    }
    d_ff_enc_dec_value = model_params["d_model_enc_dec"] * 4
    model_params["d_ff_enc_dec"] = d_ff_enc_dec_value

    fit_params = {
        "nb_ckpts":1, 
        "verbose":1
    }

    predict_params = {
        "nb_ckpts":1, 
        "scores_fname": "transformer_scores.json"
    }

    training_params = {
        "model_params": model_params,
        "fit_params": fit_params,
        "predict_params": predict_params,
        "dataset_views_to_consider": "all",
        "type_of_model": "transformer",
        "complete_dataset": False,
        "seed": trial.suggest_int("seed", 42, 1000)
    }
    training_params_global.append(training_params)
    # model = MultiomicTrainer.run_experiment(**training_params, trial=trial, output_path='/home/maoss2/scratch/optuna_test_output')
    model = MultiomicTrainer.run_experiment(trial=trial, **training_params, output_path='./')
    
    return model.trainer.callback_metrics["val_multi_acc"].item()
    # return model.trainer.callback_metrics["val_acc"].item()

def detailed_objective(trial):
    """ For the evaluation on the test set """
    all_params = training_params_global[0]
    random.seed(all_params['seed'])
    np.random.seed(all_params['seed'])
    torch.manual_seed(all_params['seed'])
    dataset = MultiomicDataset(views_to_consider=all_params["dataset_views_to_consider"], 
                                   type_of_model=all_params["type_of_model"], 
                                   complete_dataset=all_params["complete_dataset"])
    train, valid, test = multiomic_dataset_builder(dataset=dataset, test_size=0.2, valid_size=0.1)
    model = MultiomicTrainer(Namespace(**all_params['model_params']))
    scores_fname = os.path.join(all_params['fit_params']['output_path'], 
                                all_params['predict_params'].get('scores_fname', "naive_scores.txt"))
    scores = model.score(dataset=test, 
                         artifact_dir=all_params['fit_params']['output_path'], 
                         nb_ckpts=all_params['predict_params'].get('nb_ckpts', 1), 
                         scores_fname=None) # scores_fname
    return scores['acc'], scores['prec'], scores['rec'], scores['f1_score']

In [3]:
pruning = True
pruner: optuna.pruners.BasePruner = (
        optuna.pruners.MedianPruner() if pruning else optuna.pruners.NopPruner()
    )
study = optuna.create_study(direction="maximize", pruner=pruner)

[I 2021-05-21 12:13:33,799] A new study created in memory with name: no-name-5d390eba-776a-4cfa-9b50-4b81a510a62d


In [4]:
study.optimize(objective, n_trials=2, timeout=None, n_jobs=-1)

/Users/maoss2/miniconda3/lib/python3.8/site-packages/optuna/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


>>> Training configuration : >>> Training configuration : 
{
  "complete_dataset": false,
  "dataset_views_to_consider": "all",
  "fit_params": {
    "nb_ckpts": 1,
    "verbose": 1
  },
  "kwargs": {},
  "model_params": {
    "activation": "relu",
    "batch_size": 128,
    "class_weights": [
      4.1472332,
      0.87510425,
      0.30869373,
      1.2229021,
      8.47878788,
      0.7000834,
      7.94886364,
      1.87032086,
      0.63379644,
      0.63169777,
      4.19280719,
      0.40417951,
      1.08393595,
      1.90772727,
      0.72125795,
      0.87110834,
      0.59523472,
      0.61243251,
      4.38557994,
      0.63169777,
      1.94666048,
      2.04035002,
      0.67410858,
      2.08494784,
      1.40791681,
      0.79654583,
      0.74666429,
      2.74493133,
      0.65783699,
      3.02813853,
      0.65445189,
      6.6937799,
      4.76931818
    ],
    "d_ff_enc_dec": 4096,
    "d_input_enc": 2000,
    "d_model_enc_dec": 1024,
    "dropout": 0.451806705443

/Users/maoss2/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32], y=[ 0  0  0 ... 32 32 32] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


[INFO] Training


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name    | Type                     | Params
-----------------------------------------------------
0 | network | MultiomicPredictionModel | 4.4 M 
-----------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.706    Total estimated model params size (MB)


...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/maoss2/miniconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
[I 2021-05-21 12:16:55,113] Trial 0 finished with value: 2.0117878913879395 and parameters: {'lr': 5.685696833964072e-05, 'dropout': 0.370711236300861, 'weight_decay': 0.4094757073907034, 'n_heads_enc_dec': 128, 'n_layers_enc_dec': 9, 'seed': 56}. Best is trial 0 with value: 2.0117878913879395.


Validating: 0it [00:00, ?it/s]

[I 2021-05-21 12:25:52,264] Trial 1 finished with value: 5.1316304206848145 and parameters: {'lr': 0.0008713297930903779, 'dropout': 0.4518067054436615, 'weight_decay': 0.48639226155631143, 'n_heads_enc_dec': 1024, 'n_layers_enc_dec': 8, 'seed': 762}. Best is trial 1 with value: 5.1316304206848145.


In [5]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 2
Best trial:
  Value: 5.1316304206848145
  Params: 
    lr: 0.0008713297930903779
    dropout: 0.4518067054436615
    weight_decay: 0.48639226155631143
    n_heads_enc_dec: 1024
    n_layers_enc_dec: 8
    seed: 762


In [6]:
print(detailed_objective(study.best_trial))

/Users/maoss2/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32], y=[ 0  0  0 ... 32 32 32] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


./ca53af080de4e5de54cd39b81f8891bbc3deec51/checkpoints/epoch=0--val_loss=0.00.ckpt


7it [00:06,  1.05it/s]

(3.436, 6.135, 3.436, 3.553)
